In [18]:
import os

import pandas as pd

def get_max_avg_col(col_list):
    col_avg = []
    col_max = []

    for i in range(len(col_list)):
        if i == 0:
            col_avg.append(pd.NA)
            col_max.append(pd.NA)
        else:
            col_avg.append(sum(col_list[:i]) / i)
            col_max.append(max(col_list[:i])) 

    return col_avg, col_max

In [17]:
import numpy as np

def get_col_lstsq(col_list):
    col_list = pd.Series(col_list).interpolate(method='linear').bfill().ffill().tolist()
    A = []
    bias = []
    momentum = []

    for i in range(len(col_list)):
        A.append([1, i+1])
        x_lstsq, _, _, _= np.linalg.lstsq(np.array(A), np.array(col_list[:i+1]), rcond=None)
        bias.append(x_lstsq[0])
        momentum.append(x_lstsq[1])
        
    return bias, momentum

In [27]:
def get_game_stats_data_df(season_year, target_team_ids=None, target_game_date=None, training_and_testing=False):
    # Load data from SQLite table
    df = pd.read_sql_table(f"game_stats_{season_year}", "sqlite:///../database/game_stats.db")

    # Filter by season ID if training/testing mode is on
    if training_and_testing:
        season_prefix = f"2{season_year.split('-')[0]}"
        df = df[df['SEASON_ID'] == season_prefix]

    # Filter by team IDs if provided
    if target_team_ids:
        df = df[df['TEAM_ID'].isin(target_team_ids)]

    # Filter by game date if provided
    if target_game_date:
        df = df[df['GAME_DATE'] <= target_game_date]

    # Create 'HOME' binary column: 1 if 'vs.' in 'MATCHUP', else 0
    df['HOME'] = df['MATCHUP'].apply(lambda x: int('vs.' in x) if isinstance(x, str) else 0)

    features = [
        'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM',
        'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV',
        'PF', 'PTS', 'PLUS_MINUS'
    ]

    df = df.dropna()

    team_dfs = []
    for team in df['TEAM_ID'].unique():
        team_df = df[df['TEAM_ID'] == team].sort_values('GAME_DATE').copy()

        for col in features:
            col_list = team_df[col].tolist()
            avg_col, max_col = get_max_avg_col(col_list)
            bias_col, mom_col = get_col_lstsq(col_list)

            # Create new features DataFrame for this team and feature
            new_features = pd.DataFrame({
                f'{col}_BIAS': bias_col,
                f'{col}_MOM': mom_col,
                f'{col}_AVG': avg_col,
                f'{col}_MAX': max_col
            }, index=team_df.index)

            team_df = pd.concat([team_df, new_features], axis=1)

        team_dfs.append(team_df)

    teams_df = pd.concat(team_dfs, ignore_index=True)

    # Filter to only rows for the target_game_date if specified
    if target_game_date:
        teams_df = teams_df[teams_df["GAME_DATE"] == target_game_date]

    # Drop columns no longer needed
    teams_df.drop(['FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA'], axis=1, inplace=True)

    # Create opponent feature columns with suffix '_OPP'
    opp_features = [f"{col}_OPP" for col in teams_df.columns]

    home_df = teams_df[teams_df['HOME'] == 1].sort_values('GAME_ID').copy()
    away_df = teams_df[teams_df['HOME'] == 0].sort_values('GAME_ID').copy()

    away_df.columns = opp_features

    # Find common game IDs between home and away teams for proper merge
    common_game_ids = set(home_df['GAME_ID']).intersection(set(away_df['GAME_ID_OPP']))
    home_df = home_df[home_df['GAME_ID'].isin(common_game_ids)].reset_index(drop=True)
    away_df = away_df[away_df['GAME_ID_OPP'].isin(common_game_ids)].reset_index(drop=True)

    # Rename for merging on 'GAME_ID'
    away_df['GAME_ID'] = away_df['GAME_ID_OPP']

    merged_df = pd.merge(home_df, away_df, on='GAME_ID')

    # Drop duplicate or unwanted columns
    columns_to_drop = [
        'SEASON_ID_OPP', 'TEAM_ID_OPP', 'HOME_OPP', 'MIN_OPP', 'MATCHUP_OPP',
        'HOME',  # home indicator for merging purposes only
    ]
    merged_df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

    basic_features = [
        'MIN', 'FG_PCT', 'FG3_PCT', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
        'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS'
    ]
    basic_features_opp = [f"{feat}_OPP" for feat in basic_features][1:]  # skip 'MIN_OPP'?

    # Drop basic features from both sides
    merged_df.drop(columns=basic_features + basic_features_opp, inplace=True, errors='ignore')

    # Drop metadata columns that are not features
    metadata = ["SEASON_ID", "TEAM_ABBREVIATION", "TEAM_NAME", "GAME_ID", "MATCHUP"]
    metadata_opp = ["TEAM_ABBREVIATION_OPP", "TEAM_NAME_OPP", "GAME_ID_OPP"]
    merged_df.drop(columns=metadata + metadata_opp, inplace=True, errors='ignore')

    # Drop rebounding related columns explicitly
    reb_cols = [
        "REB_BIAS", "REB_MOM", "REB_AVG", "REB_MAX",
        "REB_BIAS_OPP", "REB_MOM_OPP", "REB_AVG_OPP", "REB_MAX_OPP",
        "GAME_DATE_OPP", "WL_OPP"
    ]
    merged_df.drop(columns=reb_cols, inplace=True, errors='ignore')

    # Replace Win/Loss with binary
    merged_df.replace({'L': 0, 'W': 1}, inplace=True)

    # Drop rows missing essential values
    merged_df.dropna(subset=["TEAM_ID", "MIN_AVG"], inplace=True)

    merged_df.sort_values('GAME_DATE', inplace=True)

    return merged_df

In [20]:
get_game_stats_data_df("2023-24", target_team_ids=[1610612742, 1610612760, 1610612753, 1610612749, 1610612757, 1610612758], target_game_date="2024-04-14", training_and_testing=True)

C:\Users\gthra\AppData\Local\Temp\ipykernel_1948\1698452927.py:74: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df.replace({'L': 0, 'W': 1}, inplace=True)


,TEAM_ID,GAME_DATE,WL,MIN_BIAS,MIN_MOM,MIN_AVG,MIN_MAX,FGM_BIAS,FGM_MOM,FGM_AVG,...,PF_AVG_OPP,PF_MAX_OPP,PTS_BIAS_OPP,PTS_MOM_OPP,PTS_AVG_OPP,PTS_MAX_OPP,PLUS_MINUS_BIAS_OPP,PLUS_MINUS_MOM_OPP,PLUS_MINUS_AVG_OPP,PLUS_MINUS_MAX_OPP
0,1.610613e+09,2024-04-14,1,232.847619,0.132553,238.933333,290,37.971429,0.037888,39.688889,...,19.648352,33,115.639513,0.014064,116.604396,146,2.789202,-0.017253,2.283516,38.0
1,1.610613e+09,2024-04-14,1,228.429339,0.200924,238.042553,290,40.808959,0.057489,43.457447,...,18.477778,29,114.947253,0.032919,116.800000,148,0.149158,0.042099,2.653333,50.0
2,1.610613e+09,2024-04-14,1,231.473576,0.160243,239.064516,289,39.623198,0.054965,42.225806,...,20.277778,34,106.650549,-0.019159,106.033333,137,-2.717656,-0.112062,-7.526667,40.0


In [25]:
get_game_stats_data_df("2023-24", target_team_ids=None, target_game_date=None, training_and_testing=True)

C:\Users\gthra\AppData\Local\Temp\ipykernel_1948\1698452927.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team_sorted[f'{col}_MOM'] = mom_col
C:\Users\gthra\AppData\Local\Temp\ipykernel_1948\1698452927.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team_sorted[f'{col}_AVG'] = avg_col
C:\Users\gthra\AppData\Local\Temp\ipykernel_1948\1698452927.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all c

,TEAM_ID,GAME_DATE,WL,MIN_BIAS,MIN_MOM,MIN_AVG,MIN_MAX,FGM_BIAS,FGM_MOM,FGM_AVG,...,PF_AVG_OPP,PF_MAX_OPP,PTS_BIAS_OPP,PTS_MOM_OPP,PTS_AVG_OPP,PTS_MAX_OPP,PLUS_MINUS_BIAS_OPP,PLUS_MINUS_MOM_OPP,PLUS_MINUS_AVG_OPP,PLUS_MINUS_MAX_OPP
1400,1.611661e+09,2023-05-07,1,195.000000,3.000000e+00,198.0,198,24.000000,3.000000,27.0,...,<NA>,<NA>,28.000000,28.000000,<NA>,<NA>,-12.500000,-12.500000,<NA>,<NA>
1402,1.611661e+09,2023-05-10,1,201.000000,-1.000000e+00,200.0,200,13.000000,10.000000,23.0,...,<NA>,<NA>,38.000000,38.000000,<NA>,<NA>,-6.000000,-6.000000,<NA>,<NA>
1397,1.611661e+09,2023-05-12,0,202.000000,-1.000000e+00,201.0,201,31.000000,-3.000000,28.0,...,<NA>,<NA>,45.000000,45.000000,<NA>,<NA>,9.500000,9.500000,<NA>,<NA>
1395,1.611661e+09,2023-05-13,0,197.000000,2.000000e+00,199.0,199,27.000000,-1.000000,26.0,...,22.0,25,65.666667,6.000000,75.5,81,-3.666667,6.500000,10.0,25.0
1403,1.611661e+09,2023-05-13,1,201.000000,1.339812e-14,201.0,201,13.000000,9.000000,22.0,...,18.0,18,67.000000,8.000000,75.0,75,17.000000,-12.000000,5.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1383,1.610613e+09,2024-06-06,1,234.802156,9.327657e-02,239.780952,267,41.513747,0.025488,42.914286,...,18.592593,29,117.768603,-0.052507,115.12037,148,0.575535,0.029703,2.396296,50.0
1384,1.610613e+09,2024-06-09,1,234.910245,9.027410e-02,239.792453,267,41.630400,0.022248,42.877358,...,18.568807,29,118.023019,-0.059383,114.880734,148,0.772677,0.024374,2.209174,50.0
1385,1.610613e+09,2024-06-12,0,233.506143,9.038259e-02,238.554545,264,42.025061,-0.004791,41.790909,...,16.345794,29,116.815680,0.017653,117.88785,155,8.104223,0.050347,10.884112,52.0
1386,1.610613e+09,2024-06-14,1,233.570946,8.866215e-02,238.567568,264,41.944498,-0.002652,41.756757,...,16.37037,29,117.453109,0.000269,117.777778,155,9.050866,0.024530,10.848148,52.0
